<a href="https://colab.research.google.com/github/quantumhome/DataAnalysisCaseStudy/blob/master/Netflix_Case_Study_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Problem Statement**

*   **Customer Behaviour and it’s prediction lies
at the core of every Business Model. From
Stock Exchange, e Commerce and
Automobile to even Presidential Elections,
predictions serve a great purpose. Most of
these predictions are based on the data
available about a person’s activity either
online or in person. bold text**
*   **Recommendation Engines are the much
needed manifestations of the desired
Predictability of User Activity.
Recommendation Engines move one step
further and not only give information but
put forth strategies to further increase users
interaction with the platform.**

*   **In today’s world OTT platform and Streaming
Services have taken up a big chunk in the
Retail and Entertainment industry.
Organizations like Netflix, Amazon etc.
analyse User Activity Pattern’s and suggest
products that better suit the user needs and
choices.**
*   **For the purpose of this Project we will be
creating one such Recommendation Engine
from the ground up, where every single user,
based on there area of interest and ratings,
would be recommended a list of movies that
are best suited for them.**

**Dataset Information**

  **1.ID Contains the separate keys for customer and movies.**

  **2.Rating A section contains the user ratings for all the movies.**

  **3.Genre Highlights the category of the movie.**

  **4.Movie Name Name of the movie with respect to the movie id.**

**Objective**

  **1.Find out the list of most popular and liked genre.**

  **2.Create Model that finds the best suited Movie for one user in every genre.**

  **3.Find what Genre Movies have received the best and worst ratings based on User Rating.**





In [3]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
combied_path = '/content/sample_data/combined_data_1.txt'
records = []
movie_id = None

with open(combied_path, 'r') as file:
    for line in file:
        line = line.strip()
        if line.endswith(':'):
            movie_id = int(line.replace(':', ''))
        else:
            try:
                user_id, rating, date = line.split(',')
                records.append([int(user_id), movie_id, float(rating), date])
            except:
                continue

df = pd.DataFrame(records, columns=['UserID', 'MovieID', 'Rating', 'Date'])


print("✅ Ratings loaded:", df.shape)
df.head()

✅ Ratings loaded: (24257553, 4)


,UserID,MovieID,Rating,Date
0,1488844,1,3.0,2005-09-06
1,822109,1,5.0,2005-05-13
2,885013,1,4.0,2005-10-19
3,30878,1,4.0,2005-12-26
4,823519,1,3.0,2004-05-03


# Load movie title

In [14]:
titles_path = "/content/sample_data/movie_titles.csv"

titles_df = pd.read_csv(
    titles_path,
    header=None,
    names=["MovieID", "Year", "MovieName"],
    encoding="latin-1",
    quotechar='"',
    on_bad_lines='skip'  # ✅ skips problematic lines like line 72
)

print("✅ Titles loaded:", titles_df.shape)
titles_df.head()

✅ Titles loaded: (17434, 3)


,MovieID,Year,MovieName
0,1,2003.0,Dinosaur Planet
1,2,2004.0,Isle of Man TT 2004 Review
2,3,1997.0,Character
3,4,1994.0,Paula Abdul's Get Up & Dance
4,5,2004.0,The Rise and Fall of ECW


Cleaned csv

In [15]:
df.to_csv("netflix_cleaned.csv", index=False)
print("📁 Saved to netflix_cleaned.csv")

📁 Saved to netflix_cleaned.csv


In [16]:
print(df.columns)

Index(['UserID', 'MovieID', 'Rating', 'Date'], dtype='object')


In [17]:
import random

# Define a genre pool
genre_pool = ['Action', 'Comedy', 'Drama', 'Romance', 'Thriller', 'Sci-Fi', 'Horror', 'Documentary']

# Assign genres randomly based on MovieID
random.seed(42)
df['Genre'] = df['MovieID'].map(lambda x: random.choice(genre_pool))

# Confirm assignment
df[['MovieID', 'MovieName', 'Genre']].head()


KeyError: "['MovieName'] not in index"

**Most rated genre**

In [1]:
plt.figure(figsize=(10,5))
sns.countplot(data=df, x='Genre', order=df['Genre'].value_counts().index)
plt.title('Most Rated (Popular) Genres')
plt.xlabel('Genre')
plt.ylabel('Number of Ratings')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

NameError: name 'plt' is not defined